In [1]:
from time import sleep

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging
from beamngpy.sensors import Mesh, Damage, Camera

In [2]:
bng = BeamNGpy("localhost", 25252, home='/home/vaderpc/BeamNG.tech.v0.36.4.0', debug=True)
# bng.open(launch=True)
# bng = BeamNGpy("localhost", 25252)
bng.open(None, listem_ip='*', launch=False)

In [ ]:
vehicle = Vehicle("ego_vehicle", model="rockbouncer", license="DHILO", color="White")
vehicle.attach_sensor('damage', Damage())
scenario = Scenario("smallgrid", "broken_mesh_test", description="Mesh Analysis")
# scenario.add_vehicle(
#         vehicle, pos=(408.36, -336.56, 35.54), rot_quat=(0,0, 0, 0.17)
#     )

scenario.add_vehicle(
        vehicle, pos=(0, 0, 3), rot_quat=(0,0, 0.99, 3.0)
    )
scenario.make(bng)
bng.settings.set_deterministic(60)
bng.scenario.load(scenario)
bng.scenario.start()
bng.step(60)


In [ ]:
vehicle.send_damage([
        ('front_left_suspension', 1.0, (0,0,0)),
    ])

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("front_left_suspension")')

In [ ]:
vehicle.queue_lua_command('obj:setTireBurst(2, true)')

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("engine")')

In [ ]:
vehicle.get_part_config()

In [ ]:
vehicle.get_part_config()

In [ ]:
for k, v in pairs(beamstate.breakGroup) do print(k) end

In [ ]:
vehicles = bng.get_available_vehicles()
vehicles

In [ ]:
vehicle.queue_lua_command('beamstate:breakGroup("wheel_FL")')

In [ ]:
vehicle.get_center_of_gravity()

In [ ]:
vehicle.recover()

In [ ]:
vehicle.queue_lua_command('obj:breakAllBreakGroups()')

In [ ]:
print(vehicle.is_connected)

In [ ]:
bng.step(80)

In [ ]:
vehicle.queue_lua_command('print("Hello from Lua!")')

In [ ]:
camera = Camera(
    "front_cam",
    bng,
    vehicle,
    requested_update_time = 0.1,
    is_using_shared_memory=True,
    pos=(-0.3, 1, 2),
    dir=(0, -1, 0), 
    field_of_view_y = 90,
    near_far_planes=(0.1, 1000),
    resolution=(1024, 1024),
    is_streaming=True,
    is_render_annotations=True,
    is_render_instance=True,
    is_render_depth=True,
)

In [10]:
for i in range(41):
    bng.control.step(10)

    raw_readings = camera.stream_raw()

    if i % 10 == 0:
        print("Showing images from the stream...")

        images = camera.stream()

        if camera.is_render_colours:
            images["colour"].show()
        if camera.is_render_annotations:
            images["annotation"].show()
        if camera.is_render_depth:
            images["depth"].show()

Showing images from the stream...
Showing images from the stream...
Showing images from the stream...



(eog:107276): EOG-CRITICAL **: 21:17:23.769: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:107276): GLib-GIO-CRITICAL **: 21:17:23.769: g_file_equal: assertion 'G_IS_FILE (file1)' failed


Showing images from the stream...



(eog:107276): EOG-CRITICAL **: 21:17:26.257: eog_job_thumbnail_cb: assertion 'EOG_IS_LIST_STORE (data)' failed

(eog:107276): EOG-CRITICAL **: 21:17:26.257: eog_job_thumbnail_cb: assertion 'EOG_IS_LIST_STORE (data)' failed


Showing images from the stream...


In [72]:
vehicle.queue_lua_command('energyStorage.getStorage("mainTank"):setRemainingRatio(0.1)')

In [22]:
vehicle.queue_lua_command('damageTracker.setDamage("engine", "radiatorLeak", true)')

In [6]:
# vehicle.queue_lua_command('for k,v in pairs(damageTracker) do print(k) end')
vehicle.queue_lua_command('for group, items in pairs(damageData) do for name, val in pairs(items) do print(group, name, val) end end')


In [13]:
damage_pairs = [
    ('engine', 'radiatorLeak'),
    ('engine', 'oilLeak'),
    ('engine', 'brokenStarter'),
    ('engine', 'seized'),
    ('engine', 'engineFire'),
    ('fuel', 'leak'),
    ('brakes', 'brakeFluidLeak'),
    ('transmission', 'clutchFailure'),
    ('drivetrain', 'rearDifferential'),
]

for group, name in damage_pairs:
    vehicle.queue_lua_command(f'print({group})')
    vehicle.queue_lua_command(f'damageTracker.setDamage("{group}", "{name}", true)')
    bng.step(120)

In [ ]:
vehicle.queue_lua_command('groups.process()')

In [ ]:
vehicle.queue_lua_command('meshes.processFlexbodies()')

In [57]:
vehicle.queue_lua_command('damageTracker.setDamage("engine", "radiatorLeak", true)')

In [ ]:
vehicle.get_part_config()

In [15]:
vehicle = Vehicle('ego', model='etk800')
vehicle.connect(bng)

In [17]:
a = vehicle.poll_sensors()

In [32]:
vehicle.queue_lua_command('powertrain.breakDevice(powertrain.getDevice("wheelaxleFL"))')

In [39]:
vehicle.queue_lua_command('if wheels.wheels[0] then wheels.wheels[0]:setTireBurst(1) end')

In [65]:
lua_code1= """
damageTracker.setDamage("engine", "radiatorLeak", true)
energyStorage.getStorage("mainTank"):setRemainingRatio(0.8)
"""
lua_code = """
for sys, tbl in pairs(damageTracker.damage) do
  for dtype,_ in pairs(tbl) do
    log('I','damageDump', sys..' → '..dtype)
  end
end

"""

vehicle.queue_lua_command(lua_code1)

In [66]:
vehicle.queue_lua_command('print("howdy partner")')

In [11]:
lua_code = r'''
-- list every active damage flag in the current vehicle
local function dumpBreakGroups()
  local n = obj:getBreakGroupCount()
  for i = 0, n - 1 do
    log('I','BG', string.format("%03d  %s", i, obj:getBreakGroupName(i)))
  end
end
dumpBreakGroups()
'''

vehicle.queue_lua_command(lua_code)


In [13]:
lua = r'''
-- tear off both suspensions and burst every tyre
local groups = {
  "rockbouncer_suspension_F_IFS",
  "rockbouncer_suspension_R"
}
for _,g in ipairs(groups) do
  local n = obj:getBreakGroupCount()
  for i=0,n-1 do
    if obj:getBreakGroupName(i) == g then
      obj:breakGroupIndex(i)    -- Tech-safe
    end
  end
end
for _,w in ipairs(wheels.wheels) do
  w:setTireBurst(1)
end
'''
vehicle.queue_lua_command(lua)


In [15]:
lua_command = 'obj:getBeamCount(); local beams = obj:getBeams(); return beams'
beams = vehicle.queue_lua_command(lua_command)
print(beams)

None


In [16]:
vehicle.data.beams

AttributeError: 'Vehicle' object has no attribute 'data'

In [19]:
lua_code = r'''
local cid2name = {}
for name,c in pairs(beamstate.nodeNameMap) do cid2name[c] = name end

local total = obj:getNodeCount()
for cid = 0, total-1 do
    local p = obj:getNodePosition(cid)               -- always returns vec3
    log('I','nodeDump',
        string.format("%03d  %-12s  (%.3f, %.3f, %.3f)",
            cid,
            cid2name[cid] or "<unnamed>",
            p.x, p.y, p.z))
end
'''
vehicle.queue_lua_command(lua_code)

In [29]:
vehicle.queue_lua_command('for k,v in pairs(powertrain.getDevices()) do print(k) end')

In [34]:
vehicle.queue_lua_command('beamstate.getPartDamageData()')

In [44]:
lua = r'''
for _, partType in pairs(partTypeData) do
   local split = split(partType, ":")
   if split[1] == "jbeam" and split[2] == "breakGroup" then
     local breakGroupName = split[3]
     table.insert(partBreakGroups, breakGroupName)
   end
end

local breakGroupCount = #partBreakGroups
local numberOfBrokenBreakGroups = breakGroupCount * (1 - integrityValue)
local wholeNumberOfBrokenBreakGroups = floor(numberOfBrokenBreakGroups)
local shuffledBreakGroups = arrayShuffle(partBreakGroups)
for i = 1, wholeNumberOfBrokenBreakGroups do
   table.insert(integrity.jbeam.brokenBreakGroups, shuffledBreakGroups[i])
end
vehicle.queue_lua_command(lua)
'''

vehicle.queue_lua_command(lua)


In [42]:
lua = r'''
print(obj:getPosition().z)
'''

vehicle.queue_lua_command(lua)

In [5]:
lua_list_groups = r'''
-- STEP 1: list all break-groups with their index
local n = obj.getBreakGroupCount and obj:getBreakGroupCount() or 0
log('I','BG', 'Total break-groups = '..tostring(n))

for i = 0, n - 1 do
  local name = obj:getBreakGroupName(i)
  log('I','BG', string.format("%03d  %s", i, name))
end
'''

vehicle.queue_lua_command(lua_list_groups)



In [6]:
lua_probe = r'''
-- STEP 1 (bis):  check what functions actually exist
local function p(key,val) log('I','DBG', key..' = '..tostring(val)) end

p("obj.getBreakGroupCount" , obj and obj.getBreakGroupCount)
p("obj.breakGroupIndex"    , obj and obj.breakGroupIndex)
p("beamstate"              , beamstate)
p("beamstate.breakGroupName", beamstate and beamstate.breakGroupName)
p("beamstate.breakGroupCount", beamstate and beamstate.breakGroupCount)
'''

vehicle.queue_lua_command(lua_probe)


In [7]:
lua_dump_bs = r'''
-- STEP 2: list all keys present in the current beamstate table
for k, _ in pairs(beamstate) do
  log('I','BSKEY', k)
end
'''

vehicle.queue_lua_command(lua_dump_bs)

In [8]:
vehicle.queue_lua_command('beamstate.breakAllBreakgroups()')

In [10]:
lua_list_bgroups = r'''
-- STEP 2: enumerate actual break-group names
local tbl = beamstate.deformGroupDamage or {}
local cnt = 0
for name,_ in pairs(tbl) do
  cnt = cnt + 1
  log('I','BGRP', string.format("%03d  %s", cnt, name))
end
log('I','BGRP', "Total groups found: "..cnt)
'''

vehicle.queue_lua_command(lua_list_bgroups)



In [11]:
lua_list_nodes = r'''
-- prints every node index + name + position
for id, nd in pairs(v.data.nodes) do
  log('I','NODE', string.format("%03d  %-25s  (%.2f, %.2f, %.2f)",
      id, nd.name, nd.pos.x, nd.pos.y, nd.pos.z))
end
'''

vehicle.queue_lua_command(lua_list_nodes)


In [17]:
node = 0
impulse = (0, 0, 40000)
lua_command = f'obj:applyForce({node}, {{x={impulse[0]}, y={impulse[1]}, z={impulse[2]}}})'
print(lua_command)
vehicle.queue_lua_command(lua_command)

obj:applyForce(0, {x=0, y=0, z=40000})


In [14]:
lua_check = r'''
log('I','CHK', 'addImpulse = '..tostring(obj.addImpulse))
'''

vehicle.queue_lua_command(lua_check)

In [19]:
lua_command = 'return obj:getNodeName(0)'
node_name = vehicle.queue_lua_command(lua_command)
print(node_name)

None


In [20]:
lua_command = 'local beams = obj:getBeamCount(); local beam_list = {}; for i=0, beams-1 do table.insert(beam_list, i) end; return beam_list'
beam_ids = vehicle.queue_lua_command(lua_command)
print(beam_ids)

None


In [21]:
beam_id = 42
lua_command = f'v.data:beamBreak({beam_id})'
vehicle.queue_lua_command(lua_command)

In [5]:
lua_cmd = 'beamstate.breakBreakGroup("suspension_FR")'
vehicle.queue_lua_command(lua_cmd)

In [18]:
lua_command = r'''
local beamID = nil
for _, beam in pairs(v.data.beams) do
    if beam.name ~= nil then
        print(beam.name)
        print(beam.cid)
    end
end
'''
vehicle.queue_lua_command(lua_command)

In [22]:
lua_cmd = r'''
obj:applyForceVector(1624, vec3(300000,40,10000))
'''
vehicle.queue_lua_command(lua_cmd)

In [30]:
save_path = "/tmp/rockbouncer_crash.vstate"
vehicle.queue_lua_command(f'beamstate.save("{save_path}")')